In [1]:
!pip install psycopg2

In [2]:
import pandas
import configparser
import psycopg2

In [3]:
config = configparser.ConfigParser()
config.read('config.ini')

host = config['myaws']['host']
user = config['myaws']['user']
pwd = config['myaws']['pwd']
database = config['myaws']['database']

In [4]:
conn = psycopg2.connect(
                        host = host,
                        user = user,
                        password = pwd,
                        dbname = database
)

In [5]:
cur = conn.cursor()

# Q1

In [6]:
sql_statement = '''
                select * from student_gp1
                '''

In [7]:
df = pandas.read_sql_query(sql_statement, conn)
df[:]

,email,name,major
0,johnsonks@dukes.jmu.edu ...,Kim Johnson ...,IA ...
1,jonesjm@dukes.jmu.edu ...,Jake Jones ...,IA ...
2,thomastc@dukes.jmu.edu ...,Tim Thomas ...,IA ...
3,harpersj@dukes.jmu.edu ...,Sarah Harper ...,IA ...
4,longcn@dukes.jmu.edu ...,Claire Long ...,IA ...


# Q2

In [8]:
sql_statement = '''
                select * from enroll_gp1
                '''

In [9]:
df = pandas.read_sql_query(sql_statement, conn)
df[:]

,semail,cname
0,johnsonks@dukes.jmu.edu ...,IA 200 ...
1,harpersj@dukes.jmu.edu ...,IA 240 ...
2,jonesjm@dukes.jmu.edu ...,IA 210 ...
3,longcn@dukes.jmu.edu ...,IA 210 ...
4,thomastc@dukes.jmu.edu ...,IA 261 ...


# Q3

In [10]:
sql_statement = '''
                select
                cname, count(semail)
                from enroll_gp1
                group by cname
                order by count(semail) desc
                limit 1
                '''

In [11]:
df = pandas.read_sql_query(sql_statement, conn)
df[:]

,cname,count
0,IA 210 ...,2


# Q4

In [12]:
sql_statement = '''
                select
                teacher_gp1.name as tname,
                course_gp1.name as cname
                from teacher_gp1
                inner join course_gp1 on teacher_gp1.email = course_gp1.temail
                '''

In [13]:
df = pandas.read_sql_query(sql_statement, conn)
df[:]

,tname,cname
0,Stephen Marrin ...,IA 200 ...
1,Kathleen Moore ...,IA 240 ...
2,Timothy Walton ...,IA 261 ...
3,Qingjiu Tao ...,IA 210 ...
4,Frank Harper ...,IA245 ...
5,Frank Harper ...,IA 241 ...


# Q5

In [14]:
sql_statement = '''
                select
                teacher_gp1.name as tname,
                count(course_gp1.name) as course_num
                from teacher_gp1
                inner join course_gp1 on teacher_gp1.email = course_gp1.temail
                group by teacher_gp1.name
                order by course_num desc
                limit 1
                '''

In [15]:
df = pandas.read_sql_query(sql_statement, conn)
df[:]

,tname,course_num
0,Frank Harper ...,2


# Q6

In [16]:
sql_insert_statement = '''
                        insert into teacher_gp1(email,name,office)
                        values('{}','{}', '{}')
                        '''.format('new_email1','new_name1', 'new_office1')

In [17]:
cur.execute(sql_insert_statement)

In [18]:
df = pandas.read_sql_query('''select * from teacher_gp1''', conn)
df[:]

,email,name,office
0,marrinsp@jmu.edu ...,Stephen Marrin ...,isat210
1,moore8ka@jmu.edu ...,Kathleen Moore ...,isat115
2,waltontr@jmu.edu ...,Timothy Walton ...,isat310
3,taoqx@jmu.edu ...,Qingjiu Tao ...,isat312
4,frharper@dukes.jmu ...,Frank Harper ...,isat234
5,smithjn@jmu.edu ...,John Smith ...,isat222
6,new_email1 ...,new_name1 ...,new_office1


In [19]:
sql_modify_statement = '''
                        update course_gp1
                        set temail = 'smithjn@jmu.edu'
                        where temail = 'taoqx@jmu.edu'
                        '''

In [20]:
cur.execute(sql_modify_statement)

In [21]:
sql_modify_statement = '''
                        delete from teacher_gp1
                        where email = 'taoqx@jmu.edu'
                        '''

In [22]:
cur.execute(sql_modify_statement)

In [23]:
df = pandas.read_sql_query('''select * from course_gp1''', conn)
df[:]

,name,room,temail
0,IA 200 ...,ENGEO 1210,marrinsp@jmu.edu ...
1,IA 240 ...,ENGEO 1208,moore8ka@jmu.edu ...
2,IA 261 ...,ENGEO 1208,waltontr@jmu.edu ...
3,IA245 ...,EnGeo 1204,frharper@dukes.jmu ...
4,IA 241 ...,CS 140,frharper@dukes.jmu ...
5,IA 210 ...,ENGEO 2010,smithjn@jmu.edu ...
